# Data Generation
此notebook主要用于使用文心大模型产生文本数据以便于进一步分析或者作为训练蒸馏数据，欢迎大家使用自己使用自己的开源模型向文心一言蒸馏。

In [45]:
import erniebot,os,math,time
from typing import List
erniebot.api_type = 'aistudio'
erniebot.access_token = '<EB_ACCESS_TOKEN>'

In [25]:
def transform_list(s):
    md_content = s.split('```')[1]
    md_content = md_content[md_content.find('\n'):]
    md_content = md_content.replace('\n','').replace(' ','')
    return md_content[md_content.find('['):]

def get_data(query:str) -> str:
    chat_message = [
        {'role': 'user', 'content': f"{query}，结果请用python的列表返回"}
    ]
    response = erniebot.ChatCompletion.create(model='ernie-bot-4', 
                                            messages=chat_message)

    return eval(transform_list(response.get_result()))

wenxin_reviews = get_data("请帮我生成二十条关于百度文心一言产品的用户评论")
map_reviews = get_data("请帮我生成二十条关于百度地图的用户评论")
wenku_reviews = get_data("请帮我生成二十条关于百度网盘的用户评论")
trans_reviews = get_data("请帮我生成二十条关于百度翻译的用户评论")

In [58]:
import pandas as pd
df = pd.concat(
    [
        pd.DataFrame({'reviews':wenxin_reviews,'type':['wenxin' for _ in range(len(wenxin_reviews))]}),
        pd.DataFrame({'reviews':map_reviews,'type':['map' for _ in range(len(wenxin_reviews))]}),
        pd.DataFrame({'reviews':wenku_reviews,'type':['wangpan' for _ in range(len(wenxin_reviews))]}),
        pd.DataFrame({'reviews':trans_reviews,'type':['translation' for _ in range(len(wenxin_reviews))]}),  
    ]
).reset_index(drop = True)

df.sample(10)

,reviews,type
40,百度网盘非常方便，我可以随时随地访问我的文件。,wangpan
65,我觉得百度翻译的界面很简洁，使用起来很顺畅。,translation
36,地图的夜间模式很舒适，晚上使用时不会刺眼。,map
43,百度网盘的安全措施做得很好，我可以放心地保存我的文件。,wangpan
70,百度翻译对于日常交流来说足够好了，但我不会用它来翻译重要的商业文档。,translation
48,我觉得免费用户的存储空间有点小，但付费用户可以获得更多的存储空间。,wangpan
34,百度地图的海外地图数据也很全，适合出国旅行时使用。,map
49,百度网盘是一个很好的云存储服务，我会继续使用它。,wangpan
50,百度网盘的文件分类和整理功能非常出色。,wangpan
1,我使用百度文心一言的感受是，它的自然语言处理能力非常强大。,wenxin


# Embedding
将生成的数据向量化，详细操作请参考[Embedding](03-Embedding.ipynb)

In [46]:
def get_embedding(word: List[str]) -> List[float]:
    if len(word) <= 16:
        embedding = erniebot.Embedding.create(
                                            model = 'ernie-text-embedding',
                                            input = word
                                            ).get_result()
    else:
        size = len(word)
        embedding = []
        for i in range(math.ceil(size / 16)):
            embedding.extend(erniebot.Embedding.create(model = 'ernie-text-embedding', input = word[i*16:(i+1)*16]).get_result())
            time.sleep(1) # 防止访问频繁报错
    return embedding

reviews_embedding = get_embedding(df.reviews.to_list())

assert len(reviews_embedding) == df.shape[0] 'mismatch'

将生成的embedding存储为npy文件以便使用

In [59]:
import numpy as np
np.save('./data/review_embedding.npy',np.array(reviews_embedding))
df.to_csv('./data/reviews.csv',index = False)